# INM713 SWT & KG Coursework Dataset Analysis

In [120]:
import re
import pandas as pd
import spacy
from fuzzywuzzy import process, fuzz

In [121]:
%%capture
!python -m spacy download en

In [122]:
sp = spacy.load('en_core_web_sm')

In [137]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [123]:
df = pd.read_csv("INM713_coursework_data_pizza_8358_1_reduced.csv")

## Preprocessing
### Column name tidy

In [124]:
df.columns = df.columns.str.replace(" ", "_").str.lower()

In [125]:
df.head(1)

,name,address,city,country,postcode,state,categories,menu_item,item_value,currency,item_description
0,Little Pizza Paradise,Cascade Village Mall Across From Target,Bend,US,97701,OR,Pizza Place,Bianca Pizza,22.5,USD,NaN


### Create index column

In [126]:
df = df.reset_index()
df.head(1)

,index,name,address,city,country,postcode,state,categories,menu_item,item_value,currency,item_description
0,0,Little Pizza Paradise,Cascade Village Mall Across From Target,Bend,US,97701,OR,Pizza Place,Bianca Pizza,22.5,USD,NaN


### String matching within column for menu items

<https://towardsdatascience.com/fuzzywuzzy-find-similar-strings-within-one-column-in-a-pandas-data-frame-99f6c2a0c212>

<https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings/31823872>

In [127]:
# Extract menu_item for processing
df_menu_items = df[["index", "menu_item"]]
df_menu_items.head(10)

,index,menu_item
0,0,Bianca Pizza
1,1,Cheese Pizza
2,2,"Pizza, Margherita"
3,3,"Pizza, Mushroom"
4,4,"Pizza, Puttenesca"
5,5,"Pizza, Salami Piccante"
6,6,"Pizza, White Truffle Meat Sauce"
7,7,Cheese Pizza
8,8,Hawaiian Pizza
9,9,Meat Lover Pizza


### Tidy up swapped around names
Instead of using partial token set ratio matches in fuzzywuzzy - a bit expensive on processing

In [138]:
def clean_swapped_pizza(x):
    x = x.lower()
    swapped_pizza_pattern = re.compile(r"^pizza\s?,\s?[a-z\s]+.$")
    # Matches `pizza, something asdjhgas asjdhgasd`
    if re.search(swapped_pizza_pattern, x):

        match =  re.search(swapped_pizza_pattern, x)
        result = " ".join(list(map(str.strip, match.group(0).split(",")))[::-1])
        return result
    else:
        return x

df_menu_items_cleaned = df_menu_items.copy(deep=True)
df_menu_items_cleaned["menu_item_cleaned"] = df_menu_items_cleaned["menu_item"].apply(lambda x: clean_swapped_pizza(x))

In [144]:
df_menu_items_cleaned.sample(10)

,index,menu_item,menu_item_cleaned
777,777,Medium 6 Topping Pizza,medium 6 topping pizza
2181,2181,Fajita Pizza,fajita pizza
1498,1498,Pizza Hero,pizza hero
208,208,Steak Pizza,steak pizza
2503,2503,One Topping Cheese Pizza,one topping cheese pizza
1833,1833,Bacon Cheeseburger Pizza,bacon cheeseburger pizza
29,29,Pizza Bagel With Mushroom,pizza bagel with mushroom
377,377,Four Pig Pizza,four pig pizza
3483,3483,Personal Pizza,personal pizza
2832,2832,Vegetrian Pizza,vegetrian pizza


In [154]:
unique_menu_items = df_menu_items_cleaned["menu_item_cleaned"].unique().tolist()
len(unique_menu_items) # 1596

1596

In [152]:
# " pizza" intentionally has a space preceding because we don't care about menu items JUST specificying "pizza"
df_menu_items_cleaned["menu_item_cleaned"][df_menu_items_cleaned["menu_item_cleaned"].str.lower().str.endswith(" pizza")].value_counts().nlargest(15)

cheese pizza             136
white pizza               75
margherita pizza          64
hawaiian pizza            54
bbq chicken pizza         47
pepperoni pizza           44
buffalo chicken pizza     44
taco pizza                34
veggie pizza              34
greek pizza               30
mexican pizza             29
sicilian pizza            24
vegetarian pizza          21
create your own pizza     20
build your own pizza      18
Name: menu_item_cleaned, dtype: int64

In [ ]:
# This above list is too long and we COULD do some clustering or cross-matching on similarity, but let's work smart.
# Let's take the most popular occurrences in this column from the EDA.
# cheese pizza             136
# white pizza               75
# margherita pizza          64
# hawaiian pizza            54
# bbq chicken pizza         47
# pepperoni pizza           44
# buffalo chicken pizza     44
# taco pizza                34
# veggie pizza              34
# greek pizza               30
# mexican pizza             29
# sicilian pizza            24
# vegetarian pizza          21

We focus on white, margherita and sicilian pizzas.



In [164]:
# ["margherita pizza", "pizza bianca", "white pizza", "sicilian pizza", "cheese pizza"]

is_pizza_mask = df_menu_items_cleaned["menu_item_cleaned"].str.endswith("pizza")
margherita_mask = df_menu_items_cleaned["menu_item_cleaned"].str.startswith("marg")
bianca_mask = df_menu_items_cleaned["menu_item_cleaned"].str.endswith("bianca")
sicilian_mask = df_menu_items_cleaned["menu_item_cleaned"].str.contains("sicilian")
cheese_mask = df_menu_items_cleaned["menu_item_cleaned"].str.contains("cheese pizza")

df_menu_items_cleaned_selected = df_menu_items_cleaned[is_pizza_mask & (margherita_mask | bianca_mask | sicilian_mask | cheese_mask)]

In [208]:
unique_menu_items_cleaned_selected = df_menu_items_cleaned_selected["menu_item_cleaned"].unique().tolist()

# # Create tuples of pizza names, matched pizza names, and the score
score_sort = ((umi,) + i
             for umi in unique_menu_items_cleaned_selected 
             for i in process.extract(umi, unique_menu_items_cleaned_selected, scorer=fuzz.token_set_ratio))

In [209]:
score_list = list(score_sort)

score_list.sort(key=lambda x: x[2], reverse=True) 
score_list


 cheese pizza', 100),
 ('lotti cheese pizza', 'cheese pizza', 100),
 ('lotti cheese pizza', 'lotti cheese pizza', 100),
 ('roasted vegetable and goat cheese pizza', 'cheese pizza', 100),
 ('roasted vegetable and goat cheese pizza', 'goat cheese pizza', 100),
 ('roasted vegetable and goat cheese pizza',
  'roasted vegetable and goat cheese pizza',
  100),
 ('artisan cheese pizza', 'cheese pizza', 100),
 ('artisan cheese pizza', 'artisan cheese pizza', 100),
 ('pepperoni or cheese pizza', 'cheese pizza', 100),
 ('pepperoni or cheese pizza', 'small pepperoni or cheese pizza', 100),
 ('pepperoni or cheese pizza', 'pepperoni or cheese pizza', 100),
 ('pepperoni or cheese pizza', 'pepperoni cheese pizza', 100),
 ('pan cheese pizza', 'cheese pizza', 100),
 ('pan cheese pizza', 'pan cheese pizza', 100),
 ('stuffed cheese pizza', 'cheese pizza', 100),
 ('stuffed cheese pizza', 'cheesesteak stuffed cheese pizza', 100),
 ('stuffed cheese pizza', 'stuffed cheese pizza', 100),
 ('kids cheese pizza'

In [214]:
unique_matches = set(list(zip(*score_list))[0])

In [217]:
unique_matches

{'1 large cheese pizza',
 '2 large cheese pizza',
 '2 medium cheese pizza',
 '2 small cheese pizza',
 '3 cheese pizza',
 '8in mini cheese pizza',
 'additional toppings cheese pizza',
 'adult cheese pizza',
 'artisan cheese pizza',
 'broad street cheese pizza',
 'buffalo chicken sicilian pizza',
 'calabria special sicilian pizza',
 'carolina goat cheese pizza',
 'cheese pizza',
 'cheese sicilian pizza',
 'cheesesteak stuffed cheese pizza',
 'chicken parmigiana sicilian pizza',
 'classic cheese pizza',
 'crispy thin cheese pizza',
 'crooked cheese pizza',
 'customer appreciation large cheese pizza',
 'deep dish cheese pizza',
 'eggplant parmigiana sicilian pizza',
 'five cheese pizza',
 'flatbread cheese pizza',
 'four cheese pizza',
 'giant cheese pizza',
 'goat cheese pizza',
 'grandma sicilian pizza',
 'great tasting cheese pizza',
 'honey blue cheese pizza',
 'individual cheese pizza',
 'italian pepperoni four cheese pizza',
 'jumbo cheese pizza',
 'kids cheese pizza',
 'large cheese